Autores:
- Hernán Francisco Díaz Farías
- María Angélica Parra Mendoza
- Loreto Constanza Zavala Gaete

# Tarea 1
### Realice una exploración de datos para ver que datos existen, cuales de ellos entregan información relevante para el problema y cuales no, eliminación (si corresponde) de datos nulos y duplicados, entre otros.

#### Librerías a utilizar

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
from fcmeans import FCM

#### Cargar el archivo csv

In [2]:
# Cargar el archivo csv
df = (
    pd.read_csv(
        '../data/apartments_for_rent_classified_100K.csv',
        encoding='ISO-8859-1',
        sep=';',
        low_memory=False # Para evitar warnings de dtype
        )
    )

#### Visualización de los datos

In [1]:
df.head()

NameError: name 'df' is not defined

In [4]:
df.dtypes

id                object
category          object
title             object
body              object
amenities         object
bathrooms         object
bedrooms          object
currency          object
fee               object
has_photo         object
pets_allowed      object
price            float64
price_display     object
price_type        object
square_feet       object
address           object
cityname          object
state             object
latitude         float64
longitude         object
source            object
time             float64
dtype: object

Los campos tipo object representan campos que pueden contener registros nulos o errores de escritura. Se revisará el campo **id** para empezar.
### ID

In [5]:
df['id'].isnull().sum()

0

Creamos una función para revisar si el campo **id** puede ser numérico o no.

In [6]:
def check_id(id):
    try:
        int(id)
        return True
    except:
        return False

numericos = df['id'].apply(check_id).sum()
no_numericos = df['id'].shape[0] - numericos
print(f'Numericos: {numericos}, No numéricos: {no_numericos}')


Numericos: 99823, No numéricos: 3


Se observa que hay 3 registros que no son numéricos. Se revisarán estos registros.

In [7]:
df[~df['id'].apply(check_id)]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
8808,ange of on-site community amenities. Coming so...,Gym,1,1,USD,No,Thumbnail,"Cats,Dogs",1380,"$1,38",...,NaN,Lawrence,NJ,402.659,-746.959,RentDigs.com,1.568782e+09,NaN,NaN,NaN
33780,xpansive rooftop entertainment deck and many p...,"Parking,Patio/Deck",1,1,USD,No,Thumbnail,NaN,2150,"$2,15",...,NaN,Los Angeles,CA,340.432,-1.182.509,RentDigs.com,1.568745e+09,NaN,NaN,NaN
75947,"ellite,Clubhouse,Dishwasher,Garbage Disposal,G...",2,2,USD,No,Thumbnail,"Cats,Dogs",1019,"$1,02",Monthly,...,Henrico,VA,376.249,-774.948,RentDigs.com,1550831790,NaN,NaN,NaN,NaN


Se procederá a eliminar estos registros, ya que no aportan información relevante.

In [8]:
df = df[df['id'].apply(check_id)]

Se convierte el campo **id** a tipo numérico.

In [9]:
df['id'] = df['id'].astype(int)

OverflowError: Python int too large to convert to C long

### Revisar campos nulos

In [ ]:
df.isnull().sum()

id                   0
category             0
title                0
body                 0
amenities        16077
bathrooms           66
bedrooms           127
currency             3
fee                  3
has_photo            3
pets_allowed     60634
price                4
price_display        4
price_type           3
square_feet          3
address          91880
cityname           305
state              305
latitude            28
longitude           28
source               3
time                 3
dtype: int64

In [ ]:
df[df['amenities'].isnull()]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1,1,USD,No,Thumbnail,...,"$2,20",Monthly,542,507 509 Esplanade,Redondo Beach,CA,338.520,-1.183.759,RentLingo,1.577360e+09
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3,USD,No,Thumbnail,...,"$1,25",Monthly,1500,146 Lochview Dr,Newport News,VA,370.867,-764.941,RentLingo,1.577360e+09
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2,3,USD,No,Thumbnail,...,"$1,40",Monthly,1650,3101 Morningside Dr,Raleigh,NC,358.230,-786.438,RentLingo,1.577360e+09
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1,2,USD,No,Thumbnail,...,"$1,60",Monthly,820,209 Aegean Way,Vacaville,CA,383.622,-1.219.712,RentLingo,1.577360e+09
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1,1,USD,No,Thumbnail,...,$975,Monthly,624,4805 Marquette NE,Albuquerque,NM,351.038,-1.066.110,RentLingo,1.577360e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99796,5121227775,housing/rent/apartment,"Indian Trail, 3,980 sq. ft. - in a great area.",If you re looking to rent a new home in the Br...,NaN,3.5,6,USD,No,Yes,...,"$2,20",Monthly,3980,NaN,Indian Trail,NC,350.978,-806.294,RentDigs.com,1.544186e+09
99797,5121227723,housing/rent/apartment,"Apartment in quiet area, spacious with big kit...",If you re looking to rent a new home in the Ri...,NaN,2.5,4,USD,No,Yes,...,"$1,85",Monthly,2932,NaN,Indian Trail,NC,350.978,-806.294,RentDigs.com,1.544186e+09
99811,5121223692,housing/rent/apartment,Live the you've always wanted without compromi...,"At Apartments, our apartment residences are de...",NaN,1,1,USD,No,Yes,...,$685,Monthly,625,NaN,Houston,TX,296.151,-951.998,RentDigs.com,1.544186e+09
99813,5121223111,housing/rent/apartment,Live the you've always wanted without compromi...,"At Apartments, our apartment residences are de...",NaN,1,2,USD,No,Yes,...,$870,Monthly,798,NaN,Houston,TX,296.151,-951.998,RentDigs.com,1.544186e+09


### Revisar duplicados

In [ ]:
# Revisar si hay valores duplicados
df.duplicated().sum()

84

Se remueven los registros duplicados.

In [ ]:
df = df.drop_duplicates()
df.shape

(99739, 22)

In [ ]:
df['category'].unique()

array(['housing/rent/apartment', 'housing/rent/home',
       'housing/rent/short_term', 'housing/rent', 'housing/rent/condo',
       'housing/rent/other', 'housing/rent/commercial/retail'],
      dtype=object)

In [ ]:
df_apt = df[df['category'] == 'housing/rent/apartment']

In [ ]:
df.count()

id               99739
category         99739
title            99739
body             99739
amenities        83672
bathrooms        99673
bedrooms         99612
currency         99736
fee              99736
has_photo        99736
pets_allowed     39155
price            99735
price_display    99735
price_type       99736
square_feet      99736
address           7942
cityname         99434
state            99434
latitude         99711
longitude        99711
source           99736
time             99736
dtype: int64

In [ ]:
df_apt.count()

id               99678
category         99678
title            99678
body             99678
amenities        83613
bathrooms        99613
bedrooms         99551
currency         99675
fee              99675
has_photo        99675
pets_allowed     39149
price            99674
price_display    99674
price_type       99675
square_feet      99675
address           7939
cityname         99373
state            99373
latitude         99650
longitude        99650
source           99675
time             99675
dtype: int64

In [ ]:
df.isnull().sum()

id                   0
category             0
title                0
body                 0
amenities        16067
bathrooms           66
bedrooms           127
currency             3
fee                  3
has_photo            3
pets_allowed     60584
price                4
price_display        4
price_type           3
square_feet          3
address          91797
cityname           305
state              305
latitude            28
longitude           28
source               3
time                 3
dtype: int64

In [ ]:
df_apt.isnull().sum()

id                   0
category             0
title                0
body                 0
amenities        16065
bathrooms           65
bedrooms           127
currency             3
fee                  3
has_photo            3
pets_allowed     60529
price                4
price_display        4
price_type           3
square_feet          3
address          91739
cityname           305
state              305
latitude            28
longitude           28
source               3
time                 3
dtype: int64

In [ ]:
df = df.filter(items=['bathrooms','bedrooms', 'pets_allowed', 'price', 'price_type', 'square_feet', 'cityname'], axis=1)

In [ ]:
df.count()

bathrooms       99673
bedrooms        99612
pets_allowed    39155
price           99735
price_type      99736
square_feet     99736
cityname        99434
dtype: int64

In [ ]:
df.isnull().sum()

bathrooms          66
bedrooms          127
pets_allowed    60584
price               4
price_type          3
square_feet         3
cityname          305
dtype: int64

In [ ]:
df = df.dropna(subset=['bathrooms', 'bedrooms','price', 'cityname'])
df.isnull().sum()

bathrooms           0
bedrooms            0
pets_allowed    60271
price               0
price_type          0
square_feet         0
cityname            0
dtype: int64

In [ ]:
#df['pets_allowed'] = df['pets_allowed'].fillna(value=None)
df['pets_allowed'] = df['pets_allowed'].apply(lambda x:None if pd.isnull(x) else x)

In [ ]:
if df['pets_allowed'].isin(['Cats' or 'Dogs' or 'Cats,Dogs']).any():
    df['pets_allowed'] = 1
else: df['pets_allowed'] = 0

In [ ]:
df.isnull().sum()

bathrooms       0
bedrooms        0
pets_allowed    0
price           0
price_type      0
square_feet     0
cityname        0
dtype: int64

In [26]:
df.count()

bathrooms       99251
bedrooms        99251
pets_allowed    99251
price           99251
price_type      99251
square_feet     99251
cityname        99251
dtype: int64